# 第4回課題
内包表記，並列処理

In [2]:
#!pip install joblib

In [3]:
from joblib import Parallel, delayed
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

## 内包表記
python における高速化テクの1つである内包表記の実装（for 文の削減）

### 課題1

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    for j in range(4):
        data.append(i*j)
data
```
出力：
[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

In [4]:
[i*j for i in range(5) for j in range(4)]

[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

### 課題2

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    inner = []
    for j in range(i, 6):
        inner.append(i)
    data.append(inner)
data
```
出力：
[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

In [5]:
[[i for j in range(i, 6)] for i in range(5)]

[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

## 並列処理
python における高速化テクの1つである並列化の実装（複数コア，CPU の使用）

joblib だけでなく multiprocessing もある

### 例：2乗の場合

In [6]:
%timeit 3 # 3回下記のコードを実行した時の時間を表示
r = []
for i in range(10000):
    r.append(i ** 2)
r[:3] + r[-3:]

13.8 ns ± 0.392 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


[0, 1, 4, 99940009, 99960004, 99980001]

In [7]:
?Parallel

In [8]:
%timeit 3 
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=3)( [delayed(power)(i) for i in range(10000)] )
r[:3] + r[-3:]

14.3 ns ± 0.292 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 9280 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:    1.4s finished


[0, 1, 4, 99940009, 99960004, 99980001]

### 課題1

x = [(i, i+1) for i in range(100000)] としたとき，各 index 毎に積を求めよ

出力 [0, 2, 6,（中略） 9999500006, 9999700002, 9999900000]

In [9]:
prod = lambda x: x[0] * x[1]
r = Parallel(n_jobs=-1, verbose=3)( [delayed(prod)((i, i+1)) for i in range(100000)] )
r[:3] + r[-3:]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 94578 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 100000 out of 100000 | elapsed:    3.0s finished


[0, 2, 6, 9999500006, 9999700002, 9999900000]

### 発展課題
C = np.cumsum(np.arange(1000)) の各項の差を求めよ

出力[1, 2, 3,（中略） 997, 998, 999] = np.arange(1, 1000)

In [11]:
C = np.cumsum(np.arange(1000))
sub = lambda x: x[0][x[1]+1] - x[0][x[1]] ## ここは汚いので，一応 def で再定義
def sub(inpu):
    return c[i+1] - c[i]
r = Parallel(n_jobs=-1, verbose=3)( [delayed(sub)((C, i)) for i in range(1000-1)] )
r[:3] + r[-3:]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


TypeError: sub() missing 1 required positional argument: 'i'